### First task: Gensim model for English Dataset

In [1]:
from IPython.display import display, HTML
import pandas as pd
import numpy as np
from gensim.models import FastText

model = FastText.load_fasttext_format('cc.en.300.bin')

C:\Users\han\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  


* Example application of doesn't match function

In [2]:
print(model.wv.doesnt_match('pause halt freeze shirt'.split(" ")))

shirt


C:\Users\han\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


<IPython.core.display.Javascript object>

In [588]:
engs = pd.read_csv("four_words_english_cap_3.csv", encoding= 'utf-8')
print('Main dataframe shape:', engs.shape)
homographs = pd.read_csv("four_words_english_homographs.csv", encoding= 'utf-8')
print('homographs shape:', homographs.shape)
proper_nouns = pd.read_csv("four_words_english_capital.csv", encoding= 'utf-8')
print('proper_nouns shape:', proper_nouns.shape)
double_words = pd.read_csv("four_words_english_double.csv", encoding= 'utf-8')
print('double_words shape:', double_words.shape)
display(HTML(engs.to_html()))

Main dataframe shape: (201, 4)
homographs shape: (41, 4)
proper_nouns shape: (21, 4)
double_words shape: (12, 4)


,word1,word2,word3,word4
0,train,car,plane,exercise
1,brother,sister,father,iron
2,stool,chair,couch,door
3,tree,grass,bush,clock
4,poster,picture,painting,camera
5,weight,height,length,meter
6,worm,earthworm,cockroach,tiger
7,cereals,milk,butter,knife
8,lunch,dinner,breakfast,kitchen
9,basketball,baseball,football,strategy


> Our dataset has 201 rows. 41 rows of dataset are Homographs, 21 rows of dataset are Proper Nouns and 12 rows of dataset are Double Words.

In [589]:
engs.name = 'main_dataframe'
homographs.name = 'homographs'
proper_nouns.name = 'proper_nouns'
double_words.name = 'double_words'
dfs = [engs, homographs, proper_nouns, double_words]
def calculate_accuracy_fasttext(df):
    total = 0
    correct = 0
    predict = 0
    fasttext_predictions = []
    for index, row in df[['word1', 'word2', 'word3', 'word4']].iterrows():
        predict = model.wv.doesnt_match(row.values)
        fasttext_predictions.append(predict)
        total += 1
        if predict == row["word4"]:
            correct += 1
    prediction_accuracy = correct/total
    print('Fasttext accuracy for ',df.name,' is: ', prediction_accuracy)
    df['fasttext'] = fasttext_predictions
    accuracy = str("fasttext_accuracy_{:.2f}%".format((100*prediction_accuracy)))
    return df.name, accuracy
result_dict = {}
for df in dfs:
    dataframe_name, accuracy = calculate_accuracy_fasttext(df)
    result_dict[dataframe_name] = [accuracy]
print(result_dict)

Fasttext accuracy for  main_dataframe  is:  0.8308457711442786
Fasttext accuracy for  homographs  is:  0.7560975609756098
Fasttext accuracy for  proper_nouns  is:  0.9047619047619048
Fasttext accuracy for  double_words  is:  0.0
{'main_dataframe': ['fasttext_accuracy_83.08%'], 'homographs': ['fasttext_accuracy_75.61%'], 'proper_nouns': ['fasttext_accuracy_90.48%'], 'double_words': ['fasttext_accuracy_0.00%']}


## Knowledge Graphs

In [586]:
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import display, HTML
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

endpoint = "https://query.wikidata.org/sparql"
sparql = """
SELECT ?lexeme ?representation {{
   ?lexeme ontolex:lexicalForm / ontolex:representation "WORD"@en ;
           ontolex:lexicalForm / ontolex:representation ?representation .
}}
"""
df = get_sparql_dataframe(endpoint, sparql)
df.head(None)

,lexeme,representation


In [587]:
# Script for creating a list data structure to obtain the other forms of words. (Lexeme)
import time
lexi = []
for index, row in engs.iterrows():
    for word in row.values:
        sparql = sparql.replace("WORD", word)
        #time.sleep(1)
        lex = list(get_sparql_dataframe(endpoint, sparql).representation.unique())
        index = 0
        while(index < len(lex)):
            if lex[index] not in model.wv.vocab:
                del lex[index]
                index -= 1
            index += 1
        if word not in lex:
            lex.append(word)
        else:
            pass
        lexi.append(lex)
        sparql = """
        SELECT ?lexeme ?representation {{
           ?lexeme ontolex:lexicalForm / ontolex:representation "WORD"@en ;
                   ontolex:lexicalForm / ontolex:representation ?representation .
        }}
        """

KeyboardInterrupt: 

In [ ]:
# Save it to disk.
import pickle
with open("list_lex_final.txt", "wb") as fp:   
    pickle.dump(lexi, fp)

In [590]:
# Read it from disk
with open("list_lex_final.txt", "rb") as fp:   
    lexi = pickle.load(fp)
lexi

[['train', 'trains', 'trained', 'training'],
 ['car', 'cars'],
 ['plane', 'planes', 'planed', 'planing', 'planer'],
 ['exercise', 'exercises', 'exercised', 'exercising'],
 ['exercise', 'exercises', 'exercised', 'exercising'],
 ['brother', 'brothers', 'bretheren'],
 ['sister', 'sisters'],
 ['father', 'fathers', 'fathered', 'fathering'],
 ['iron', 'irons', 'ironed', 'ironing'],
 ['iron', 'irons', 'ironed', 'ironing'],
 ['stool'],
 ['chair', 'chairs', 'chaired', 'chairing'],
 ['couch', 'couches', 'couched', 'couching'],
 ['door', 'doors'],
 ['door', 'doors'],
 ['tree', 'trees', 'treed', 'treeing'],
 ['grass', 'grasses'],
 ['bush', 'bushes'],
 ['clock', 'clocks', 'clocked', 'clocking'],
 ['clock', 'clocks', 'clocked', 'clocking'],
 ['poster', 'posters'],
 ['picture', 'pictures', 'pictured', 'picturing'],
 ['paint', 'paints', 'painted', 'painting', 'paintings'],
 ['camera', 'cameras'],
 ['camera', 'cameras'],
 ['weight', 'weights', 'weighted', 'weighting'],
 ['height', 'heights'],
 ['length

In [ ]:
lexi_dic = {}
for index, row in engs.iterrows():
    for word in row.values:
        sparql = sparql.replace("WORD", word)
        if word in lexi_dic.keys():
            pass
        else:
            #time.sleep(1)
            lexi_dic[word] = list(get_sparql_dataframe(endpoint, sparql).representation.unique())
            if bool(lexi_dic[word]) == False:
                lexi_dic[word] = [word]
            index = 0
            while(index < len(lexi_dic[word])):
                if lexi_dic[word][index] not in model.wv.vocab:
                    del lexi_dic[word][index]
                    index -= 1
                index += 1
            
        sparql = """
        SELECT ?lexeme ?representation {{
           ?lexeme ontolex:lexicalForm / ontolex:representation "WORD"@en ;
                   ontolex:lexicalForm / ontolex:representation ?representation .
        }}
        """

In [ ]:
# save it to disk
import pickle
with open("dict_lex_final.txt", "wb") as fp:   #Pickling
    pickle.dump(lexi_dic, fp)

In [591]:
# Read it from disk
with open("dict_lex_final.txt", "rb") as fp:   # Unpickling
    lexi_dic = pickle.load(fp)
lexi_dic

{'train': ['train', 'trains', 'trained', 'training'],
 'car': ['car', 'cars'],
 'plane': ['plane', 'planes', 'planed', 'planing', 'planer'],
 'exercise': ['exercise', 'exercises', 'exercised', 'exercising'],
 'brother': ['brother', 'brothers', 'bretheren'],
 'sister': ['sister', 'sisters'],
 'father': ['father', 'fathers', 'fathered', 'fathering'],
 'iron': ['iron', 'irons', 'ironed', 'ironing'],
 'stool': ['stool'],
 'chair': ['chair', 'chairs', 'chaired', 'chairing'],
 'couch': ['couch', 'couches', 'couched', 'couching'],
 'door': ['door', 'doors'],
 'tree': ['tree', 'trees', 'treed', 'treeing'],
 'grass': ['grass', 'grasses'],
 'bush': ['bush', 'bushes'],
 'clock': ['clock', 'clocks', 'clocked', 'clocking'],
 'poster': ['poster', 'posters'],
 'picture': ['picture', 'pictures', 'pictured', 'picturing'],
 'painting': ['paint', 'paints', 'painted', 'painting', 'paintings'],
 'camera': ['camera', 'cameras'],
 'weight': ['weight', 'weights', 'weighted', 'weighting'],
 'height': ['height'

In [592]:
from IPython.display import Javascript
display(Javascript('IPython.notebook.execute_cells_below()'))

<IPython.core.display.Javascript object>

In [593]:
lexi_nd = []
for index, row in engs[['word1', 'word2', 'word3', 'word4']].iterrows():
    words = row.values
    for word in words:
        counter = 0
        for row in lexi:
            if word in row:
                counter += 1
                print(word, row)
                lexi_nd.append(row)
                if counter > 0:
                    break    
print(lexi_nd) # lexi_nd list doesnt have the duplicates

train ['train', 'trains', 'trained', 'training']
car ['car', 'cars']
plane ['plane', 'planes', 'planed', 'planing', 'planer']
exercise ['exercise', 'exercises', 'exercised', 'exercising']
brother ['brother', 'brothers', 'bretheren']
sister ['sister', 'sisters']
father ['father', 'fathers', 'fathered', 'fathering']
iron ['iron', 'irons', 'ironed', 'ironing']
stool ['stool']
chair ['chair', 'chairs', 'chaired', 'chairing']
couch ['couch', 'couches', 'couched', 'couching']
door ['door', 'doors']
tree ['tree', 'trees', 'treed', 'treeing']
grass ['grass', 'grasses']
bush ['bush', 'bushes']
clock ['clock', 'clocks', 'clocked', 'clocking']
poster ['poster', 'posters']
picture ['picture', 'pictures', 'pictured', 'picturing']
painting ['paint', 'paints', 'painted', 'painting', 'paintings']
camera ['camera', 'cameras']
weight ['weight', 'weights', 'weighted', 'weighting']
height ['height', 'heights']
length ['length', 'lengths']
meter ['meter', 'meters', 'metered', 'metering', 'metre', 'metres']

study ['study', 'studies', 'studied', 'studying']
drink ['drink', 'drinks', 'drank', 'drunk', 'drinking']
drinking ['drink', 'drinks', 'drank', 'drunk', 'drinking']
drink ['drink', 'drinks', 'drank', 'drunk', 'drinking']
sand ['sand', 'sands', 'sanded', 'sanding']
tea ['tea', 'teas']
coffee ['coffee', 'coffees']
cocoa ['cocoa', 'cocoas']
cancer ['cancer', 'cancers']
heart ['heart', 'hearts']
lung ['lung', 'lungs']
kidney ['kidney', 'kidneys']
headache ['headache', 'headaches']
Denmark ['Denmark']
Norway ['Norway']
Sweden ['Sweden']
Germany ['Germany']
sleep ['sleep', 'sleeps', 'slept', 'sleeping']
sleeping ['sleep', 'sleeps', 'slept', 'sleeping']
slept ['sleep', 'sleeps', 'slept', 'sleeping']
school ['school', 'schools', 'schooled', 'schooling']
fly ['fly', 'flies', 'flew', 'flown', 'flying']
spider ['spider', 'spiders']
ant ['ant', 'ants']
bird ['bird', 'birds']
carrot ['carrot', 'carrots']
tomato ['tomato', 'tomatoes']
cucumber ['cucumber']
car ['car', 'cars']
sun ['sun', 'suns', 'su

In [594]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
from sklearn.decomposition import PCA

"""
t-Distributed Stochastic Neighbor Embedding
The t-SNE algorithm calculates a similarity measure between pairs of instances in the high dimensional space and 
in the low dimensional space. It then tries to optimize these two similarity measures using a cost function. 
"""
def reduce_dimensions_tsne(lexi):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for ls in lexi:
        for word in ls:
            vectors.append(model.wv.word_vec(word))
            labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)

    # reduce using t-SNE
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [595]:
x_vals, y_vals, labels = reduce_dimensions_tsne(lexi_nd)

def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go
    import plotly.graph_objects as go
    
    layout = go.Layout(autosize=False, width=850, height=550, margin=dict(l=50, r=50, b=100, t=100, pad=4))
    trace = go.Scatter(x=x_vals, y=y_vals, mode='text+markers', text=labels, textposition="top center")
    fig = go.Figure(data=[trace], layout=layout)
    init_notebook_mode(connected=True)
    iplot(fig, filename='word-embedding-plot')
        
plot_with_plotly(x_vals, y_vals, labels)

In [596]:
from IPython.display import Image
Image(url="https://github.com/SukruHan/AdvancedMachineLearning/blob/master/image1.PNG?raw=true")

In [597]:
def reduce_dimensions_tsne_norm(lexi):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for ls in lexi:
        for word in ls:
            vectors.append(model.wv.word_vec(word, use_norm=True))
            labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)

    # reduce using t-SNE
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

x_vals, y_vals, labels = reduce_dimensions_tsne_norm(lexi_nd)
plot_with_plotly(x_vals, y_vals, labels)

In [598]:
Image(url="https://github.com/SukruHan/AdvancedMachineLearning/blob/master/image2.PNG?raw=true")

* We can observe that, after normalizing the vectors, it is better uniformly distirbuted instead of creating a density in the center
* Now let's visualize fewer words.

In [599]:
x_vals, y_vals, labels = reduce_dimensions_tsne_norm(lexi_nd[0:4])
plot_with_plotly(x_vals, y_vals, labels)

In [600]:
Image(url="https://github.com/SukruHan/AdvancedMachineLearning/blob/master/image3.PNG?raw=true")

* Train and trains are closer to plane and cars however trained and training are more closer to exercise and exercising. (After limiting our data, it is easier to see.)

* From now on, we will examine a less complex example such as "brother", "sister", "father", "iron".

In [601]:
x_vals, y_vals, labels = reduce_dimensions_tsne_norm(lexi_nd[4:8])
plot_with_plotly(x_vals, y_vals, labels)

In [602]:
Image(url="https://github.com/SukruHan/AdvancedMachineLearning/blob/master/image4.PNG?raw=true")

Let's visualize with the mean values of each lexeme.

In [646]:
def reduce_dimensions_tsne_norm_with_mean_labels(lexi):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)
    means_vector = []
    means_label = []
    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for ls in lexi:
        counter = 0
        for word in ls:
            vectors.append(model.wv.word_vec(word, use_norm=True))
            labels.append(word)
            index = 0
            temp = []
            while(index<len(ls)):
                temp.append(model.wv.word_vec(word, use_norm=True))
                if counter < 1:
                    means_label.append("{words}_mean".format(words = word))
                counter += 1
                index += 1
            mean = np.mean(temp, axis=0)
        means_vector.append(mean)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)
    
    means_vector = np.asarray(means_vector)
    means_label = np.asarray(means_label)
    
    # concatenating the arrays; 
    
    vectors2 = np.concatenate([vectors,means_vector])
    labels2 = np.concatenate([labels,means_label])

    # reduce using t-sne
    vectors2 = np.asarray(vectors2)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors2 = tsne.fit_transform(vectors2)
    
    x_vals = [v[0] for v in vectors2]
    y_vals = [v[1] for v in vectors2]
    
    sh = len(vectors2)
    means_x_vals = [v[0] for v in vectors2[-sh:]]
    means_y_vals = [v[1] for v in vectors2[-sh:]]
    
    return x_vals, y_vals, labels2, means_label, means_x_vals, means_y_vals

In [647]:
x_vals, y_vals, labels, means_label, means_x_vals, means_y_vals  = reduce_dimensions_tsne_norm_with_mean_labels(lexi_nd[4:8])
plot_with_plotly(x_vals, y_vals, labels)

In [605]:
Image(url="https://github.com/SukruHan/AdvancedMachineLearning/blob/master/image5.PNG?raw=true")

### We can see that, mean values doesn't make much sense with t-SNE method. Now, we will try with PCA.

In [642]:
def reduce_dimensions_pca_norm_with_mean_labels(lexi):
    
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)
    means_vector = []
    means_label = []
    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for ls in lexi:
        counter = 0
        for word in ls:
            vectors.append(model.wv.word_vec(word, use_norm=True))
            labels.append(word)
            index = 0
            temp = []
            while(index<len(ls)):
                temp.append(model.wv.word_vec(word, use_norm=True))
                if counter < 1:
                    means_label.append("{words}_mean".format(words = word))
                counter += 1
                index += 1
            mean = np.mean(temp, axis=0)
        means_vector.append(mean)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)
    
    means_vector = np.asarray(means_vector)
    means_label = np.asarray(means_label)
    
    # concatenating the arrays; 
    
    vectors2 = np.concatenate([vectors,means_vector])
    labels2 = np.concatenate([labels,means_label])

    # reduce using PCA
    vectors2 = np.asarray(vectors2)
    pca = PCA(n_components=num_dimensions, random_state=0)
    vectors2 = pca.fit_transform(vectors2)
    
    x_vals = [v[0] for v in vectors2]
    y_vals = [v[1] for v in vectors2]
    
    sh = len(means_vector)
    means_x_vals = [v[0] for v in vectors2[-sh:]]
    means_y_vals = [v[1] for v in vectors2[-sh:]]
    
    return x_vals, y_vals, labels2, means_label, means_x_vals, means_y_vals

x_vals, y_vals, labels, means_label, means_x_vals, means_y_vals  = reduce_dimensions_pca_norm_with_mean_labels(lexi_nd[4:8])
plot_with_plotly(x_vals, y_vals, labels)

In [607]:
Image(url="https://github.com/SukruHan/AdvancedMachineLearning/blob/master/image6.PNG?raw=true")

In [608]:
homographs.loc[homographs.fasttext != homographs.word4]

,word1,word2,word3,word4,fasttext
5,book,notebook,sheet,stapler,book
7,ventilator,air conditioner,fan,follower,air conditioner
12,bend,curve,bow,arrow,curve
21,attach,clinch,tie,knot,attach
25,market,bakery,butcher,bread,market
27,right,forward,left,correct,forward
33,several,many,lots,few,lots
38,summer,bikini,sand,snow,summer
39,run,sleep,drink,bottle,run
40,fly,spider,ant,bird,fly


In [609]:
for key, values in lexi_dic.items():
    if values == []:
        lexi_dic[key] = [key]

In [610]:
lexi_dic

{'train': ['train', 'trains', 'trained', 'training'],
 'car': ['car', 'cars'],
 'plane': ['plane', 'planes', 'planed', 'planing', 'planer'],
 'exercise': ['exercise', 'exercises', 'exercised', 'exercising'],
 'brother': ['brother', 'brothers', 'bretheren'],
 'sister': ['sister', 'sisters'],
 'father': ['father', 'fathers', 'fathered', 'fathering'],
 'iron': ['iron', 'irons', 'ironed', 'ironing'],
 'stool': ['stool'],
 'chair': ['chair', 'chairs', 'chaired', 'chairing'],
 'couch': ['couch', 'couches', 'couched', 'couching'],
 'door': ['door', 'doors'],
 'tree': ['tree', 'trees', 'treed', 'treeing'],
 'grass': ['grass', 'grasses'],
 'bush': ['bush', 'bushes'],
 'clock': ['clock', 'clocks', 'clocked', 'clocking'],
 'poster': ['poster', 'posters'],
 'picture': ['picture', 'pictures', 'pictured', 'picturing'],
 'painting': ['paint', 'paints', 'painted', 'painting', 'paintings'],
 'camera': ['camera', 'cameras'],
 'weight': ['weight', 'weights', 'weighted', 'weighting'],
 'height': ['height'

In [611]:
def reduce_dimensions_pca_norm_with_mean_labels_randomInput(lexi):
    
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)
    means_vector = []
    means_label = []
    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for ls in lexi:
        counter = 0
        for word in ls:
            vectors.append(model.wv.word_vec(word, use_norm=True))
            labels.append(word)
            all_forms = [i for i in lexi_dic[word]]
            all_forms_vectors = np.vstack(model.wv.word_vec(word, use_norm=True) for word in all_forms).astype('float32')
            mean_all_forms = np.mean(all_forms_vectors, axis=0)
            means_label.append("{words}_mean".format(words = all_forms[0]))
            means_vector.append(mean_all_forms)
    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)
    
    means_vector = np.asarray(means_vector)
    means_label = np.asarray(means_label)
    
    # concatenating the arrays; 
    
    vectors2 = np.concatenate([vectors,means_vector])
    labels2 = np.concatenate([labels,means_label])

    # reduce using PCA
    vectors2 = np.asarray(vectors2)
    pca = PCA(n_components=num_dimensions, random_state=0)
    vectors2 = pca.fit_transform(vectors2)
    
    x_vals = [v[0] for v in vectors2]
    y_vals = [v[1] for v in vectors2]
    
    sh = len(means_vector)
    means_x_vals = [v[0] for v in vectors2[-sh:]]
    means_y_vals = [v[1] for v in vectors2[-sh:]]
    
    return x_vals, y_vals, labels2, means_label, means_x_vals, means_y_vals, 
x_vals, y_vals, labels, means_label, means_x_vals, means_y_vals  = reduce_dimensions_pca_norm_with_mean_labels_randomInput([list(homographs.iloc[12].values)[0:4]])
plot_with_plotly(x_vals, y_vals, labels)

C:\Users\han\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



In [612]:
Image(url="https://github.com/SukruHan/AdvancedMachineLearning/blob/master/image7.PNG?raw=true")

* In this homograph example, Fasttext has determined the outlier as curve but it is actuaaly arrow. As it is obviuos from above outlier isn't detected so easily which has a reverse situation in brother, sister, father, iron example. In homograph example every word is at another corner but in the previous ex iron was explicitly out of group.

### After applying PCA it makes more sense in terms of mean vector representations of lexemes.

### What happens if we try to find outlier according to results of mean vectors of PCA and TSNE?

* This distance_func gets inputs of reduced dimension vectors of PCA or Tsne and then returns outlier according to 'cosine similarity' measure. Each distance measure is summed for each mean vector and the outlier is the one which has the greatest value.

In [643]:
def distance_func(x_vals, y_vals, labels):
    
    import numpy as np
    import itertools
    import operator
    from scipy.spatial import distance
    
    if len(x_vals) != len(y_vals):
        raise Exception("Shape of input vectors should be same.")
    
    sh = len(x_vals)
    
    mx = np.concatenate([x_vals, y_vals]).reshape(2,sh).T
    
    vector_dic = {}
    row = 0
    while(row < len(mx)):
        for i in list(labels):
            if i not in vector_dic.keys():
                vector_dic[i] = list(mx[row])
            else:
                pass
            row += 1
    
    comb_list = list(map(dict, itertools.combinations(vector_dic.items(), 2)))
    for word in vector_dic.keys():
        vector_dic[word] = 0
    
    max_comb = "None"
    max_value = float(0)
    for dicts in comb_list:
        index = 0
        while(index < len(dicts)-1):
            for word1 in vector_dic.keys():
                for word2 in dicts.keys():
                    if word1 == word2:
                        item1 = list(dicts.values())[index]
                        item2 = list(dicts.values())[index+1]
                        item1 = np.asarray(item1)
                        item2 = np.asarray(item2)
                        vector_dic[word1] += distance.cosine(item1, item2)
            if distance.cosine(list(dicts.values())[index], list(dicts.values())[index+1]) > max_value:
                max_value = distance.cosine(list(dicts.values())[index], list(dicts.values())[index+1])
                max_comb = "Comparison of {i} and {j}:".format(i=list(dicts.keys())[index], j=list(dicts.keys())[index+1])
            index += 1
    outlier = max(vector_dic.items(), key=operator.itemgetter(1))[0]
    outlier = outlier.replace('_mean', '')
    return outlier

In [644]:
distance_func(means_x_vals, means_y_vals, means_label)
# For the example of brother, sister, father and exercise, outlier according to pca is father.
# Let's apply this to all words

'father'

In [649]:
def dimesionality_reduction_outlier_calc(method_func, lexi):
    i = 0
    outlier_list = []
    while(i<len(lexi)):
        x_vals, y_vals, labels, means_label, means_x_vals, means_y_vals  = method_func(lexi[i:i+4])
        outlier = distance_func(means_x_vals, means_y_vals, means_label)
        outlier_list.append(outlier)
        i += 4
    return outlier_list

outlier_pca = dimesionality_reduction_outlier_calc(reduce_dimensions_pca_norm_with_mean_labels, lexi_nd)
outlier_tsne = dimesionality_reduction_outlier_calc(reduce_dimensions_tsne_norm_with_mean_labels, lexi_nd)

In [650]:
def dimesionality_reduction_result(method_name, outlier_list):
    false_prediction = 0
    index = 0
    while(index < len(engs.word4)):
        if engs.word4[index] != outlier_list[index]:
            false_prediction += 1
        index += 1
    accuracy =  100 * (1 - (false_prediction/len(engs.word4)))
    accuracy_str = "{}_{:.2f}%".format(method_name, accuracy)
    return accuracy_str
resul_pca = dimesionality_reduction_result('dimensionality_reduction_pca', outlier_pca)
resul_tsne = dimesionality_reduction_result('dimensionality_reduction_tsne', outlier_tsne)
result_dict["main_dataframe"].append(str(resul_pca))
result_dict["main_dataframe"].append(str(resul_tsne))
result_dict

{'main_dataframe': ['fasttext_accuracy_83.08%',
  'dimensionality_reduction_pca_47.76%',
  'dimensionality_reduction_tsne_26.37%'],
 'homographs': ['fasttext_accuracy_75.61%'],
 'proper_nouns': ['fasttext_accuracy_90.48%'],
 'double_words': ['fasttext_accuracy_0.00%']}

As we seee from the aforementioned result outlier analysis with pca and tsne is not powerful, since we lose lots of information while we reduce the dimensionality of word vectors.

### Wembedder Model (Like in Finn's notebook)

In [651]:
display(Javascript('IPython.notebook.execute_cells_below()'))

<IPython.core.display.Javascript object>

In [652]:
# Standard python modules
from functools import lru_cache
from os.path import expanduser, join

# Extra python modules
import requests

# Wembedder module
import sys
sys.path.append(expanduser("wembedder/"))
import wembedder.model

In [653]:
# Create a utility function to search with Wikidata API.

HEADERS = {'User-Agent': 'Finn Årup Nielsen, http://people.compute.dtu.dk/faan/'}

# This code has been modified from Scholia - https://github.com/fnielsen/scholia
# There is a cache to avoid searching multiple times.
# The results should have been stored if we wanted to have full reproducibility! :(
@lru_cache(maxsize=500)
def search(query, limit=10):
    """Search Wikidata.

    Parameters
    ----------
    query : str
        Query string.
    limit : int, optional
        Number of maximum search results to return.

    Returns
    -------
    result : list of dicts

    """
    params = {
        "action": "wbsearchentities",
        "search": query,
        "format": "json",
        "language": "en",
        "uselang": "en",
        "type": "item"}
    
    # Query the Wikidata API
    response = requests.get(
        "https://www.wikidata.org/w/api.php",
        params=params,
        headers=HEADERS)

    # Convert the response
    response_data = response.json()
    items = response_data['search']
    results = [
        {'q': item['title'],
         'description': item.get('description', '')}
        for item in items]

    return results

In [654]:
# Check to see whether the Wikidata API search returns something appropriate
search('exercise')

[{'q': 'Q219067',
  'description': 'bodily activity that enhances or maintains physical fitness and overall health and wellness'},
 {'q': 'Q357104',
  'description': 'employment of military resources in training for military operations'},
 {'q': 'Q29051769',
  'description': 'performance of activities in order to develop or maintain readiness or competence in various forms of endeavors'},
 {'q': 'Q35869',
  'description': 'long-term disease involving inflamed airways'},
 {'q': 'Q1178021', 'description': 'Wikipedia disambiguation page'},
 {'q': 'Q18612162', 'description': '189th strip of the webcomic xkcd'},
 {'q': 'Q5420020', 'description': ''}]

In [655]:
wembedder_model = wembedder.model.Model.load()
wembedder_model.metadata

{'filename': 'wikidata-20170613-truthy-BETA-cbow-size=100-window=1-min_count=20'}

In [656]:
# Utility function and dictionaries to convert between Wikidata Q-idenfier and word
q_to_word = {}
word_to_q = {}

def words_to_qs(words):
    qs = []
    for word in words:
        result = search(word)
        if len(result) > 0:
            q = result[0]['q']
            word_to_q[word] = q
            q_to_word[q] = word
        else:
            q = ''
        qs.append(q)
    return qs

In [657]:
dfs = [engs, homographs, proper_nouns, double_words]
def calculate_accuracy_wembedder(df):
    wembedder = []
    for idx, words in df[['word1', 'word2', 'word3', 'word4']].iterrows():
        words = words.values[:4]
        qs = words_to_qs(words)    
        # This may take some time to download from the Wikidata API
        print(qs)
        try:
            q = wembedder_model.wv.doesnt_match(qs)
            outlier = q_to_word.get(q, words[0])
        except ValueError:
            # If the 
            outlier = words[0]
        wembedder.append(outlier)
    df['wembedder'] = wembedder
    accuracy = np.mean(df.word4 == df['wembedder'])
    accuracy = str("wembedder_accuracy_{:.2f}%".format((100*accuracy)))
    return df.name, accuracy

for df in dfs:
    dataframe_name, accuracy = calculate_accuracy_wembedder(df)
    result_dict[dataframe_name].append(accuracy)
print(result_dict)

['Q870', 'Q929', 'Q197', 'Q219067']


C:\Users\han\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



['Q31184', 'Q31184', 'Q7565', 'Q677']
['Q496', 'Q15026', 'Q131514', 'Q36794']
['Q10884', 'Q43238', 'Q42295', 'Q376']
['Q429785', 'Q478798', 'Q11629', 'Q15328']
['Q11423', 'Q208826', 'Q36253', 'Q11573']
['Q180712', 'Q30092', 'Q25309', 'Q19939']
['Q30485671', 'Q8495', 'Q34172', 'Q32489']
['Q12896105', 'Q657221', 'Q80973', 'Q43164']
['Q5372', 'Q5369', 'Q2736', 'Q185451']
['Q75', 'Q18545', 'Q240502', 'Q1357']
['Q76768', 'Q39908', 'Q223269', 'Q10943']
['Q1323314', 'Q865054', 'Q321767', 'Q1368']
['Q929', 'Q43193', 'Q686', 'Q197']
['Q8486', 'Q6097', 'Q147538', 'Q178']
['Q8094', 'Q7925', 'Q37602', 'Q10294']
['Q395', 'Q12483', 'Q9492', 'Q2249676']
['Q29175', 'Q80091', 'Q169534', 'Q600134']
['Q48282', 'Q37226', 'Q37517', 'Q168983']
['Q614467', 'Q648864', 'Q50620486', 'Q14674']
['Q165447', 'Q14674', 'Q486736', 'Q37654']
['Q1527264', 'Q6821213', 'Q194489', 'Q185451']
['Q808', 'Q12136', 'Q10876', 'Q11190']
['Q244', 'Q212198', 'Q58339', 'Q12823105']
['Q3914', 'Q3918', 'Q1664720', 'Q41176']
['Q571', 

['Q1339614', 'Q210832', 'Q29615886', 'Q201413']
['Q859965', 'Q18041641', 'Q1124733', 'Q5338673']
['Q870', 'Q929', 'Q197', 'Q219067']
['Q429785', 'Q478798', 'Q11629', 'Q15328']
['Q11423', 'Q208826', 'Q36253', 'Q11573']
['Q75', 'Q18545', 'Q240502', 'Q1357']
['Q1323314', 'Q865054', 'Q321767', 'Q1368']
['Q571', 'Q43013', 'Q391788', 'Q489157']
['Q1734', 'Q42486', 'Q5369', 'Q28425']
['Q813243', 'Q1265533', 'Q193220', 'Q31888506']
['Q4610556', 'Q1630279', 'Q708', 'Q663']
['Q4817', 'Q166154', 'Q1366302', 'Q35872']
['Q207580', 'Q29545091', 'Q76469', 'Q7364']
['Q43056', 'Q1510951', 'Q4830417', 'Q12791']
['Q671288', 'Q161973', 'Q46311', 'Q45922']
['Q28877', 'Q43483', 'Q2842797', 'Q1148747']
['Q3957', 'Q515', 'Q5119', 'Q133492']
['Q80228', 'Q1207302', 'Q16', 'Q667987']
['Q212198', 'Q866742', 'Q244', 'Q19819742']
['Q152', 'Q2796766', 'Q46185', 'Q8355']
['Q15332388', 'Q1498', 'Q76074', 'Q56557591']
['Q620207', 'Q33767', 'Q2001588', 'Q1503211']
['Q25235', 'Q43297', 'Q178794', 'Q5290']
['Q22003219', '

In [658]:
wembedder_to_code = {}
for key, value in q_to_word.items():
    wembedder_to_code[value] = [key]
wembedder_to_code # we have the words as the keys, codes as the keys

{'train': ['Q870'],
 'car': ['Q929'],
 'plane': ['Q197'],
 'exercise': ['Q219067'],
 'sister': ['Q31184'],
 'father': ['Q7565'],
 'iron': ['Q677'],
 'stool': ['Q496'],
 'chair': ['Q15026'],
 'couch': ['Q131514'],
 'door': ['Q36794'],
 'tree': ['Q10884'],
 'grass': ['Q43238'],
 'bush': ['Q42295'],
 'clock': ['Q376'],
 'poster': ['Q429785'],
 'picture': ['Q478798'],
 'painting': ['Q11629'],
 'camera': ['Q15328'],
 'weight': ['Q11423'],
 'height': ['Q208826'],
 'length': ['Q36253'],
 'meter': ['Q11573'],
 'worm': ['Q180712'],
 'earthworm': ['Q30092'],
 'cockroach': ['Q25309'],
 'tiger': ['Q19939'],
 'cereals': ['Q30485671'],
 'milk': ['Q8495'],
 'butter': ['Q34172'],
 'knife': ['Q32489'],
 'lunch': ['Q12896105'],
 'dinner': ['Q657221'],
 'breakfast': ['Q80973'],
 'kitchen': ['Q43164'],
 'basketball': ['Q5372'],
 'baseball': ['Q5369'],
 'football': ['Q2736'],
 'strategy': ['Q185451'],
 'net': ['Q75'],
 'ball': ['Q18545'],
 'racket': ['Q240502'],
 'spider': ['Q1357'],
 'shirt': ['Q76768'],


In [659]:
def word_to_code_converter(word):
    return wembedder_to_code[word][0]

def reduce_dimensions_pca_wembedder(input_):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)
    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for ls in input_:
        for word in ls:
            code = word_to_code_converter(word)
            vectors.append(wembedder_model.wv.word_vec(code, use_norm=True))
            labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)

    # reduce using PCA
    vectors = np.asarray(vectors)
    pca = PCA(n_components=num_dimensions, random_state=0)
    vectors = pca.fit_transform(vectors)
    
    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)
    
x_vals, y_vals, labels = reduce_dimensions_pca_wembedder([list(engs.iloc[0].values[0:4])])
plot_with_plotly(x_vals, y_vals, labels)

In [660]:
Image(url="https://github.com/SukruHan/AdvancedMachineLearning/blob/master/image8.PNG?raw=true")

In [661]:
engs.loc[engs.wembedder == engs.word4].tail()

,word1,word2,word3,word4,fasttext,wembedder
187,January,October,December,Juliette,Juliette,Juliette
190,monkey,elephant,tiger,Oliver,Oliver,Oliver
191,oil,salt,garlic,Scottish,Scottish,Scottish
195,top brass,administration,executive,cream,top brass,cream
196,focal point,focus,centerpiece,light,focal point,light


In [662]:
list(engs.iloc[187].values)[0:4]

['January', 'October', 'December', 'Juliette']

In [663]:
x_vals, y_vals, labels = reduce_dimensions_pca_wembedder([list(engs.iloc[187].values)[0:4]])
plot_with_plotly(x_vals, y_vals, labels)

In [664]:
Image(url="https://github.com/SukruHan/AdvancedMachineLearning/blob/master/image9.PNG?raw=true")

### Months are on a line but Juliette is far from the months so that it is an outlier detected correctly.

In [665]:
engs.loc[engs.wembedder != engs.word4].head()

,word1,word2,word3,word4,fasttext,wembedder
0,train,car,plane,exercise,exercise,car
1,brother,sister,father,iron,iron,father
3,tree,grass,bush,clock,clock,grass
4,poster,picture,painting,camera,camera,painting
5,weight,height,length,meter,meter,length


In [666]:
x_vals, y_vals, labels = reduce_dimensions_pca_wembedder([list(engs.iloc[3].values)[0:4]])
plot_with_plotly(x_vals, y_vals, labels)

In [667]:
Image(url="https://github.com/SukruHan/AdvancedMachineLearning/blob/master/image10.PNG?raw=true")

### It is weird to observe that clock is on the same line with tree and bush

In [668]:
result_dict

{'main_dataframe': ['fasttext_accuracy_83.08%',
  'dimensionality_reduction_pca_47.76%',
  'dimensionality_reduction_tsne_26.37%',
  'wembedder_accuracy_38.81%'],
 'homographs': ['fasttext_accuracy_75.61%', 'wembedder_accuracy_24.39%'],
 'proper_nouns': ['fasttext_accuracy_90.48%', 'wembedder_accuracy_61.90%'],
 'double_words': ['fasttext_accuracy_0.00%', 'wembedder_accuracy_33.33%']}

* As we can see from the results above wembedder accuracy is highest at proper nouns

In [669]:
def calculate_accuracy_fasttext_wembedder(df, upper_count_limit):
    outliers = []
    for idx, words in df.iterrows():
        upper_count = 0
        for word in words:
            if word[0].upper() == word[0]:
                upper_count += 1
        if upper_count > upper_count_limit:
            outlier = df.loc[idx, 'wembedder']
        else:
            outlier = df.loc[idx, 'fasttext']
        outliers.append(outlier)
    df['fasttext_wembedder'] = outliers
    accuracy = np.mean(df.word4 == df['fasttext_wembedder'])
    accuracy = str("fasttext_wembedder_with_upper_count_limit_{}_accuracy_{:.2f}%".format(upper_count_limit,100*accuracy))
    return df.name, accuracy

for df in dfs:
    dataframe_name, accuracy = calculate_accuracy_fasttext_wembedder(df,1)
    result_dict[dataframe_name].append(accuracy)
    dataframe_name, accuracy = calculate_accuracy_fasttext_wembedder(df,2)
    result_dict[dataframe_name].append(accuracy)
    dataframe_name, accuracy = calculate_accuracy_fasttext_wembedder(df,3)
    result_dict[dataframe_name].append(accuracy)
    dataframe_name, accuracy = calculate_accuracy_fasttext_wembedder(df,4)
    result_dict[dataframe_name].append(accuracy)
result_dict

{'main_dataframe': ['fasttext_accuracy_83.08%',
  'dimensionality_reduction_pca_47.76%',
  'dimensionality_reduction_tsne_26.37%',
  'wembedder_accuracy_38.81%',
  'fasttext_wembedder_with_upper_count_limit_1_accuracy_80.10%',
  'fasttext_wembedder_with_upper_count_limit_2_accuracy_80.60%',
  'fasttext_wembedder_with_upper_count_limit_3_accuracy_80.60%',
  'fasttext_wembedder_with_upper_count_limit_4_accuracy_80.60%'],
 'homographs': ['fasttext_accuracy_75.61%',
  'wembedder_accuracy_24.39%',
  'fasttext_wembedder_with_upper_count_limit_1_accuracy_75.61%',
  'fasttext_wembedder_with_upper_count_limit_2_accuracy_75.61%',
  'fasttext_wembedder_with_upper_count_limit_3_accuracy_75.61%',
  'fasttext_wembedder_with_upper_count_limit_4_accuracy_75.61%'],
 'proper_nouns': ['fasttext_accuracy_90.48%',
  'wembedder_accuracy_61.90%',
  'fasttext_wembedder_with_upper_count_limit_1_accuracy_61.90%',
  'fasttext_wembedder_with_upper_count_limit_2_accuracy_66.67%',
  'fasttext_wembedder_with_upper_c

* we can observe from the results that; when the number of words which starts with upper case is 2, fasttext and wembedder yields the best result.

In [670]:
def our_doesnt_match(model, words):
    import operator
    import numpy as np 
    from scipy.spatial import distance
    dict_distances = {}
    used_words = [word for word in words if word in model]
    vectors = np.vstack(model.wv.word_vec(word, use_norm=True) for word in used_words).astype('float32')
    mean = vectors.mean(axis=0).astype('float32')
    for i in range(len(used_words)):
        dict_distances[used_words[i]] = distance.cosine(vectors[i], mean)
        outlier_label = max(dict_distances.items(), key=operator.itemgetter(1))[0]
        outlier_value = max(dict_distances.items(), key=operator.itemgetter(1))[1]
    return outlier_label

In [671]:
dfs = [engs, homographs, proper_nouns, double_words]
def calculate_accuracy_our_doesnt_match(df):
    our_doesnt_match_predictions = []
    for index, row in df[['word1', 'word2', 'word3', 'word4']].iterrows():
        predict = our_doesnt_match(model, row.values)
        our_doesnt_match_predictions.append(predict)
    df['our_doesnt_match'] = our_doesnt_match_predictions        
    accuracy = np.mean(df.word4 == df['our_doesnt_match'])
    accuracy = str("our_doesnt_match_accuracy_{:.2f}%".format((100*accuracy)))
    return df.name, accuracy
for df in dfs:
    dataframe_name, accuracy = calculate_accuracy_our_doesnt_match(df)
    result_dict[dataframe_name].append(accuracy)
print(result_dict)

{'main_dataframe': ['fasttext_accuracy_83.08%', 'dimensionality_reduction_pca_47.76%', 'dimensionality_reduction_tsne_26.37%', 'wembedder_accuracy_38.81%', 'fasttext_wembedder_with_upper_count_limit_1_accuracy_80.10%', 'fasttext_wembedder_with_upper_count_limit_2_accuracy_80.60%', 'fasttext_wembedder_with_upper_count_limit_3_accuracy_80.60%', 'fasttext_wembedder_with_upper_count_limit_4_accuracy_80.60%', 'our_doesnt_match_accuracy_83.08%'], 'homographs': ['fasttext_accuracy_75.61%', 'wembedder_accuracy_24.39%', 'fasttext_wembedder_with_upper_count_limit_1_accuracy_75.61%', 'fasttext_wembedder_with_upper_count_limit_2_accuracy_75.61%', 'fasttext_wembedder_with_upper_count_limit_3_accuracy_75.61%', 'fasttext_wembedder_with_upper_count_limit_4_accuracy_75.61%', 'our_doesnt_match_accuracy_75.61%'], 'proper_nouns': ['fasttext_accuracy_90.48%', 'wembedder_accuracy_61.90%', 'fasttext_wembedder_with_upper_count_limit_1_accuracy_61.90%', 'fasttext_wembedder_with_upper_count_limit_2_accuracy_66.

C:\Users\han\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).

C:\Users\han\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



In [672]:
engs.loc[engs.fasttext != engs.our_doesnt_match]
# as you can see, our doesnt match works perfectly fine

,word1,word2,word3,word4,fasttext,wembedder,fasttext_wembedder,our_doesnt_match


### Our next question is what happens if we average the word vectors of each lexeme and then average the averages?

In [673]:
def mean_vectors_of_lexeme(lexi):
    dic_mean = dict()
    row_counter = 0
    counter_for_row = 0
    mean_value = 0
    for row in lexi:
        alls = []
        counter = 0
        for i in row:
            if counter == 0:
                word = i
                counter +=1
        vectors = np.vstack(model.wv.word_vec(word, use_norm =True) for word in row).astype('float32') #vectors are normalized
        mean_value = np.mean(vectors, axis=0)
        dic_mean.setdefault("row{}".format(row_counter), []).append([word, mean_value])
        counter_for_row += 1
        if counter_for_row % 4 == 0:
            row_counter += 1
    return dic_mean
dict_mean = mean_vectors_of_lexeme(lexi_nd)

C:\Users\han\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



In [674]:
def our_doesnt_match_with_averaging_forms(dictionary_of_mean_vectors):
    outlier = []
    for key, value in dictionary_of_mean_vectors.items():
        dict_distances = {}
        used_words = []
        used_words.append(value[0][0])
        used_words.append(value[1][0])
        used_words.append(value[2][0])
        used_words.append(value[3][0])
        vectors = np.vstack((value[0][1], value[1][1], value[2][1], value[3][1])).astype('float32')
        mean = vectors.mean(axis=0).astype('float32')
        for i in range(len(used_words)):
            dict_distances[used_words[i]] = distance.cosine(vectors[i], mean)
            outlier_label = max(dict_distances.items(), key=operator.itemgetter(1))[0]
            outlier_value = max(dict_distances.items(), key=operator.itemgetter(1))[1]
        outlier.append(outlier_label)
    return outlier
outliers_avg_forms = our_doesnt_match_with_averaging_forms(dict_mean)
engs['our_doesnt_match_with_averaging_forms'] = outliers_avg_forms
accuracy = np.mean(engs.word4 == engs['our_doesnt_match_with_averaging_forms'])
accuracy = str("our_doesnt_match_with_averaging_forms_accuracy_{:.2f}%".format((100*accuracy)))
result_dict['main_dataframe'].append(accuracy)
result_dict

{'main_dataframe': ['fasttext_accuracy_83.08%',
  'dimensionality_reduction_pca_47.76%',
  'dimensionality_reduction_tsne_26.37%',
  'wembedder_accuracy_38.81%',
  'fasttext_wembedder_with_upper_count_limit_1_accuracy_80.10%',
  'fasttext_wembedder_with_upper_count_limit_2_accuracy_80.60%',
  'fasttext_wembedder_with_upper_count_limit_3_accuracy_80.60%',
  'fasttext_wembedder_with_upper_count_limit_4_accuracy_80.60%',
  'our_doesnt_match_accuracy_83.08%',
  'our_doesnt_match_with_averaging_forms_accuracy_78.61%'],
 'homographs': ['fasttext_accuracy_75.61%',
  'wembedder_accuracy_24.39%',
  'fasttext_wembedder_with_upper_count_limit_1_accuracy_75.61%',
  'fasttext_wembedder_with_upper_count_limit_2_accuracy_75.61%',
  'fasttext_wembedder_with_upper_count_limit_3_accuracy_75.61%',
  'fasttext_wembedder_with_upper_count_limit_4_accuracy_75.61%',
  'our_doesnt_match_accuracy_75.61%'],
 'proper_nouns': ['fasttext_accuracy_90.48%',
  'wembedder_accuracy_61.90%',
  'fasttext_wembedder_with_up

###  Averaging all forms didn't improve the result.

* Now we will try to improve our doesnt match function. 'our_doesnt_match_double_words' function takes care of double words. If there is any double words, we average the word vector of each word of double words. Then we calculate the cosine distance to find out the outlier.

In [675]:
def our_doesnt_match_double_words(model, words):
    import operator
    import numpy as np 
    from scipy.spatial import distance
    dict_distances = {}
    used_words = [word for word in words if word in model]
    double_words = []
    for i in used_words:
        if len(i.split(' ')) > 1:
            double_words.append(i)
    if len(double_words)>0:
        for doubles in double_words:
            word_1 = doubles.split(' ')[0]
            word_2 = doubles.split(' ')[1]
            word_s = [word_1, word_2]
        others_words = [item for item in used_words if item not in double_words]
        vector_doubles = np.vstack(model.wv.word_vec(word, use_norm=True) for word in word_s).astype('float32')
        mean_of_doubles = np.mean(vector_doubles, axis=0)
        vector_of_others = np.vstack(model.wv.word_vec(word, use_norm=True) for word in others_words).astype('float32')
        vectors = np.vstack((vector_of_others,mean_of_doubles)).astype('float32')
        used_words = others_words + double_words
    else:
        vectors = np.vstack(model.wv.word_vec(word, use_norm=True) for word in used_words).astype('float32')
    mean = vectors.mean(axis=0).astype('float32')
    for i in range(len(used_words)):
        dict_distances[used_words[i]] = distance.cosine(vectors[i], mean)
        outlier_label = max(dict_distances.items(), key=operator.itemgetter(1))[0]
        outlier_value = max(dict_distances.items(), key=operator.itemgetter(1))[1]
    return outlier_label

In [676]:
dfs = [engs, homographs, proper_nouns, double_words]
def calculate_accuracy_our_doesnt_match_with_doubles(df):
    our_doesnt_match_predictions = []
    for index, row in df[['word1', 'word2', 'word3', 'word4']].iterrows():
        predict = our_doesnt_match_double_words(model, row.values)
        our_doesnt_match_predictions.append(predict)
    df['our_doesnt_match_with_double_words'] = our_doesnt_match_predictions        
    accuracy = np.mean(df.word4 == df['our_doesnt_match_with_double_words'])
    accuracy = str("our_doesnt_match_with_double_words_accuracy_{:.2f}%".format((100*accuracy)))
    return df.name, accuracy
for df in dfs:
    dataframe_name, accuracy = calculate_accuracy_our_doesnt_match_with_doubles(df)
    result_dict[dataframe_name].append(accuracy)
print(result_dict)

C:\Users\han\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).

C:\Users\han\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.

C:\Users\han\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.

C:\Users\han\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.1

{'main_dataframe': ['fasttext_accuracy_83.08%', 'dimensionality_reduction_pca_47.76%', 'dimensionality_reduction_tsne_26.37%', 'wembedder_accuracy_38.81%', 'fasttext_wembedder_with_upper_count_limit_1_accuracy_80.10%', 'fasttext_wembedder_with_upper_count_limit_2_accuracy_80.60%', 'fasttext_wembedder_with_upper_count_limit_3_accuracy_80.60%', 'fasttext_wembedder_with_upper_count_limit_4_accuracy_80.60%', 'our_doesnt_match_accuracy_83.08%', 'our_doesnt_match_with_averaging_forms_accuracy_78.61%', 'our_doesnt_match_with_double_words_accuracy_88.06%'], 'homographs': ['fasttext_accuracy_75.61%', 'wembedder_accuracy_24.39%', 'fasttext_wembedder_with_upper_count_limit_1_accuracy_75.61%', 'fasttext_wembedder_with_upper_count_limit_2_accuracy_75.61%', 'fasttext_wembedder_with_upper_count_limit_3_accuracy_75.61%', 'fasttext_wembedder_with_upper_count_limit_4_accuracy_75.61%', 'our_doesnt_match_accuracy_75.61%', 'our_doesnt_match_with_double_words_accuracy_78.05%'], 'proper_nouns': ['fasttext_ac

In [677]:
def our_doesnt_match_double_words_sum(model, words):
    import operator
    import numpy as np 
    from scipy.spatial import distance
    dict_distances = {}
    used_words = [word for word in words if word in model]
    double_words = []
    for i in used_words:
        if len(i.split(' ')) > 1:
            double_words.append(i)
    if len(double_words)>0:
        for doubles in double_words:
            word_1 = doubles.split(' ')[0]
            word_2 = doubles.split(' ')[1]
            word_s = [word_1, word_2]
        others_words = [item for item in used_words if item not in double_words]
        vector_doubles = np.vstack(model.wv.word_vec(word, use_norm=True) for word in word_s).astype('float32')
        sum_of_doubles = np.sum(vector_doubles, axis=0)
        vector_of_others = np.vstack(model.wv.word_vec(word, use_norm=True) for word in others_words).astype('float32')
        vectors = np.vstack((vector_of_others,sum_of_doubles)).astype('float32')
        used_words = others_words + double_words
    else:
        vectors = np.vstack(model.wv.word_vec(word, use_norm=True) for word in used_words).astype('float32')
    mean = vectors.mean(axis=0).astype('float32')
    for i in range(len(used_words)):
        dict_distances[used_words[i]] = distance.cosine(vectors[i], mean)
        outlier_label = max(dict_distances.items(), key=operator.itemgetter(1))[0]
        outlier_value = max(dict_distances.items(), key=operator.itemgetter(1))[1]
    return outlier_label

dfs = [engs, homographs, proper_nouns, double_words]
def calculate_accuracy_our_doesnt_match_with_doubles_sum(df):
    our_doesnt_match_predictions = []
    for index, row in df[['word1', 'word2', 'word3', 'word4']].iterrows():
        predict = our_doesnt_match_double_words_sum(model, row.values)
        our_doesnt_match_predictions.append(predict)
    df['our_doesnt_match_with_double_words_sum'] = our_doesnt_match_predictions        
    accuracy = np.mean(df.word4 == df['our_doesnt_match_with_double_words_sum'])
    accuracy = str("our_doesnt_match_with_double_words_sum_accuracy_{:.2f}%".format((100*accuracy)))
    return df.name, accuracy
for df in dfs:
    dataframe_name, accuracy = calculate_accuracy_our_doesnt_match_with_doubles_sum(df)
    result_dict[dataframe_name].append(accuracy)
result_dict

C:\Users\han\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).

C:\Users\han\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.

C:\Users\han\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.

C:\Users\han\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.1

{'main_dataframe': ['fasttext_accuracy_83.08%',
  'dimensionality_reduction_pca_47.76%',
  'dimensionality_reduction_tsne_26.37%',
  'wembedder_accuracy_38.81%',
  'fasttext_wembedder_with_upper_count_limit_1_accuracy_80.10%',
  'fasttext_wembedder_with_upper_count_limit_2_accuracy_80.60%',
  'fasttext_wembedder_with_upper_count_limit_3_accuracy_80.60%',
  'fasttext_wembedder_with_upper_count_limit_4_accuracy_80.60%',
  'our_doesnt_match_accuracy_83.08%',
  'our_doesnt_match_with_averaging_forms_accuracy_78.61%',
  'our_doesnt_match_with_double_words_accuracy_88.06%',
  'our_doesnt_match_with_double_words_sum_accuracy_89.05%'],
 'homographs': ['fasttext_accuracy_75.61%',
  'wembedder_accuracy_24.39%',
  'fasttext_wembedder_with_upper_count_limit_1_accuracy_75.61%',
  'fasttext_wembedder_with_upper_count_limit_2_accuracy_75.61%',
  'fasttext_wembedder_with_upper_count_limit_3_accuracy_75.61%',
  'fasttext_wembedder_with_upper_count_limit_4_accuracy_75.61%',
  'our_doesnt_match_accuracy_

### instead of averaging, summing got better results

In [678]:
def our_doesnt_match_corr(model, words):
    import operator
    import numpy as np 
    from scipy.spatial import distance
    dict_distances = {}
    used_words = [word for word in words if word in model]
    vectors = np.vstack(model.wv.word_vec(word, use_norm=True) for word in used_words).astype('float32')
    mean = vectors.mean(axis=0).astype('float32')
    for i in range(len(used_words)):
        dict_distances[used_words[i]] = distance.correlation(vectors[i], mean)
        outlier_label = max(dict_distances.items(), key=operator.itemgetter(1))[0]
        outlier_value = max(dict_distances.items(), key=operator.itemgetter(1))[1]
    return outlier_label

def calculate_accuracy_our_doesnt_match_corr(df):
    our_doesnt_match_predictions = []
    for index, row in df[['word1', 'word2', 'word3', 'word4']].iterrows():
        predict = our_doesnt_match_corr(model, row.values)
        our_doesnt_match_predictions.append(predict)
    df['our_doesnt_match_with_corr'] = our_doesnt_match_predictions        
    accuracy = np.mean(df.word4 == df['our_doesnt_match_with_corr'])
    accuracy = str("our_doesnt_match_with_corr_accuracy_{:.2f}%".format((100*accuracy)))
    return df.name, accuracy
for df in dfs:
    dataframe_name, accuracy = calculate_accuracy_our_doesnt_match_corr(df)
    result_dict[dataframe_name].append(accuracy)
result_dict

C:\Users\han\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).

C:\Users\han\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



{'main_dataframe': ['fasttext_accuracy_83.08%',
  'dimensionality_reduction_pca_47.76%',
  'dimensionality_reduction_tsne_26.37%',
  'wembedder_accuracy_38.81%',
  'fasttext_wembedder_with_upper_count_limit_1_accuracy_80.10%',
  'fasttext_wembedder_with_upper_count_limit_2_accuracy_80.60%',
  'fasttext_wembedder_with_upper_count_limit_3_accuracy_80.60%',
  'fasttext_wembedder_with_upper_count_limit_4_accuracy_80.60%',
  'our_doesnt_match_accuracy_83.08%',
  'our_doesnt_match_with_averaging_forms_accuracy_78.61%',
  'our_doesnt_match_with_double_words_accuracy_88.06%',
  'our_doesnt_match_with_double_words_sum_accuracy_89.05%',
  'our_doesnt_match_with_corr_accuracy_83.08%'],
 'homographs': ['fasttext_accuracy_75.61%',
  'wembedder_accuracy_24.39%',
  'fasttext_wembedder_with_upper_count_limit_1_accuracy_75.61%',
  'fasttext_wembedder_with_upper_count_limit_2_accuracy_75.61%',
  'fasttext_wembedder_with_upper_count_limit_3_accuracy_75.61%',
  'fasttext_wembedder_with_upper_count_limit_4

* Correlation results the same result with cosine distance function.

In [679]:
def reduce_dimensions_pca_double_words(lexi):
    
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)
    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for ls in lexi:
        counter = 0
        for word in ls:
            if len(word.split(' ')) > 1:
                word_1 = word.split(' ')[0]
                word_2 = word.split(' ')[1]
                word_s = [word_1, word_2]
                vector_doubles = np.vstack(model.wv.word_vec(word, use_norm=True) for word in word_s).astype('float32')
                sum_doubles = np.mean(vector_doubles, axis=0)
                vectors.append(sum_doubles)
            else:
                vectors.append(model.wv.word_vec(word, use_norm=True))
            labels.append(word)
    # convert both lists into numpy vectors for reduction
    #vectors_original = vectors.copy()
    labels = np.asarray(labels)
    # reduce using PCA
    vectors = np.asarray(vectors)
    pca = PCA(n_components=num_dimensions, random_state=0)
    vectors = pca.fit_transform(vectors)
    
    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    
    return x_vals, y_vals, labels #, vectors_original
x_vals, y_vals, labels = reduce_dimensions_pca_double_words([['dining room', 'dining', 'room']])

plot_with_plotly(x_vals, y_vals, labels)

C:\Users\han\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



In [680]:
Image(url="https://github.com/SukruHan/AdvancedMachineLearning/blob/master/image11.PNG?raw=true")

In [681]:
x_vals, y_vals, labels = reduce_dimensions_pca_double_words([['in front', 'in', 'front']])
plot_with_plotly(x_vals, y_vals, labels)

C:\Users\han\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



In [682]:
Image(url="https://github.com/SukruHan/AdvancedMachineLearning/blob/master/image12.PNG?raw=true")

In [684]:
result_dict

{'main_dataframe': ['fasttext_accuracy_83.08%',
  'dimensionality_reduction_pca_47.76%',
  'dimensionality_reduction_tsne_26.37%',
  'wembedder_accuracy_38.81%',
  'fasttext_wembedder_with_upper_count_limit_1_accuracy_80.10%',
  'fasttext_wembedder_with_upper_count_limit_2_accuracy_80.60%',
  'fasttext_wembedder_with_upper_count_limit_3_accuracy_80.60%',
  'fasttext_wembedder_with_upper_count_limit_4_accuracy_80.60%',
  'our_doesnt_match_accuracy_83.08%',
  'our_doesnt_match_with_averaging_forms_accuracy_78.61%',
  'our_doesnt_match_with_double_words_accuracy_88.06%',
  'our_doesnt_match_with_double_words_sum_accuracy_89.05%',
  'our_doesnt_match_with_corr_accuracy_83.08%'],
 'homographs': ['fasttext_accuracy_75.61%',
  'wembedder_accuracy_24.39%',
  'fasttext_wembedder_with_upper_count_limit_1_accuracy_75.61%',
  'fasttext_wembedder_with_upper_count_limit_2_accuracy_75.61%',
  'fasttext_wembedder_with_upper_count_limit_3_accuracy_75.61%',
  'fasttext_wembedder_with_upper_count_limit_4

In [688]:
for i in ['homographs','proper_nouns','double_words']:
    result_dict[i].append('none')
    result_dict[i].append('none')
    result_dict[i].append('none')

In [689]:
df_results = pd.DataFrame(result_dict)
df_results

,main_dataframe,homographs,proper_nouns,double_words
0,fasttext_accuracy_83.08%,fasttext_accuracy_75.61%,fasttext_accuracy_90.48%,fasttext_accuracy_0.00%
1,dimensionality_reduction_pca_47.76%,wembedder_accuracy_24.39%,wembedder_accuracy_61.90%,wembedder_accuracy_33.33%
2,dimensionality_reduction_tsne_26.37%,fasttext_wembedder_with_upper_count_limit_1_ac...,fasttext_wembedder_with_upper_count_limit_1_ac...,fasttext_wembedder_with_upper_count_limit_1_ac...
3,wembedder_accuracy_38.81%,fasttext_wembedder_with_upper_count_limit_2_ac...,fasttext_wembedder_with_upper_count_limit_2_ac...,fasttext_wembedder_with_upper_count_limit_2_ac...
4,fasttext_wembedder_with_upper_count_limit_1_ac...,fasttext_wembedder_with_upper_count_limit_3_ac...,fasttext_wembedder_with_upper_count_limit_3_ac...,fasttext_wembedder_with_upper_count_limit_3_ac...
5,fasttext_wembedder_with_upper_count_limit_2_ac...,fasttext_wembedder_with_upper_count_limit_4_ac...,fasttext_wembedder_with_upper_count_limit_4_ac...,fasttext_wembedder_with_upper_count_limit_4_ac...
6,fasttext_wembedder_with_upper_count_limit_3_ac...,our_doesnt_match_accuracy_75.61%,our_doesnt_match_accuracy_90.48%,our_doesnt_match_accuracy_0.00%
7,fasttext_wembedder_with_upper_count_limit_4_ac...,our_doesnt_match_with_double_words_accuracy_78...,our_doesnt_match_with_double_words_accuracy_90...,our_doesnt_match_with_double_words_accuracy_83...
8,our_doesnt_match_accuracy_83.08%,our_doesnt_match_with_double_words_sum_accurac...,our_doesnt_match_with_double_words_sum_accurac...,our_doesnt_match_with_double_words_sum_accurac...
9,our_doesnt_match_with_averaging_forms_accuracy...,our_doesnt_match_with_corr_accuracy_75.61%,our_doesnt_match_with_corr_accuracy_90.48%,our_doesnt_match_with_corr_accuracy_0.00%


In [690]:
df_results.to_csv('results_models.csv', index=False)